In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar dados
df = pd.read_csv(r'\data\creditcard.csv')

# Separar features e target
X = df.drop(['id', 'Class'], axis=1, errors='ignore')
y = df['Class']

# Split em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo Random Forest com tuning
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=2,
    class_weight='balanced',
    random_state=42
)

# Cross-validation
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='f1')
print("\nCross-validation F1 scores:", cv_scores)
print("Average F1 score:", np.mean(cv_scores))

# Treinar modelo
rf_model.fit(X_train_scaled, y_train)

# Previsão com probabilidades
y_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]

# ---------- Threshold ajustável ----------
threshold = 0.4  # Ajuste esse valor conforme necessário
y_pred_thresh = (y_pred_proba >= threshold).astype(int)

# Relatório de classificação com novo threshold
print("\nRelatório de Classificação com Threshold =", threshold)
print(classification_report(y_test, y_pred_thresh))

# ---------- Matriz de Confusão ----------
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_thresh)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# ---------- Gráfico Precision-Recall vs Threshold ----------
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
plt.plot(thresholds, recall[:-1], label='Recall', color='green')
plt.axhline(y=0.9, color='red', linestyle='--', label='Precision = 0.90')
plt.axhline(y=0.8, color='orange', linestyle='--', label='Recall = 0.80')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision and Recall vs Threshold')
plt.legend()
plt.grid()
plt.show()


KeyboardInterrupt: 